## Extracting Data From GitHub
### Created by Pablo Fernández Bravo

<div style="text-align: justify">En este notebook se va a tratar de experimentar con la extracción del contenido y los comentarios de un repositorio de GitHub mediante PyGithub. LA información se guardará en ficheros .json para poder utilizarlos posteriormente.</div>

<div style="text-align: justify">La posterior celda contiene la orden necesaria para instalar PyGithub directamente en Jupyter.
En caso de querer desintalar la dependencia utilizar la instrucción comentada.</div>

In [1]:
import sys
!{sys.executable} -m pip install PyGithub
# !python -m pip uninstall PyGithub --yes

In [2]:
import os
import json
import github as gh
from pprint import pprint

<div style="text-align: justify">Para poder funcionar necesitamos tener un token de acceso personal en un fichero <code>json</code> en la siguiente ubicicación <code>config\config.json</code> estructurado como se puede ver la imagen a continuación:
<img src="img/config_file.png"><div>

In [3]:
with open('config/config.json') as config_file:
    config = json.load(config_file)

### Instanciar una conexión a GitHub
Se crea una instancia de la conexión, se obtiene el repositiorio y se recoge la información de las issues.

In [4]:
g = gh.Github(config['GITHUB_TOKEN'])

In [5]:
repo = g.get_repo("MrpYA45/github-text-mining-tfg")
issues = repo.get_issues(state="all")
pprint(issues.get_page(0))

[Issue(title="Testear las limitaciones de la API de GitHub", number=4),
 Issue(title="Prototipar la descarga de issues mediante PyGithub", number=3),
 Issue(title="Explorar la organización de la arquitectura de microservicios", number=2),
 Issue(title="Investigar sobre la conexión con GitHub", number=1)]


In [6]:
pprint(issues[0].labels[0])

Label(name="test")


In [7]:
pprint(issues[-1].get_comments()[0].body)

('Recuerda que la guía de referencia es '
 'https://docs.github.com/en/rest/reference/issues\r\n'
 '@Kencho ¿puedes dejar a Pablo el código de extracción de issues y '
 'almacenamiento en BD?')


### Crear una función que permita extraer la información de las issues
A partir de la URL de un repositorio se recoge la información de sus issues y se devuelve en un diccionario. Se debe tener en cuenta que la API de GitHub posee una limitación de  [5000 peticiones por hora](https://docs.github.com/en/developers/apps/rate-limits-for-github-apps#:~:text=User%2Dto%2Dserver%20requests%20are,per%20hour%20for%20that%20user.), por lo tanto se tendrá en cuenta en el tratamiento de excepciones.

In [8]:
from urllib.parse import urlparse
import datetime as dt
import logging

def get_repo_issues(repo_dir: str) -> None:
    url = urlparse(repo_dir)
    extracted_data: dict = {
        'url': repo_dir,
        'issues': {}
    }
    repo = g.get_repo(url.path[1:])
    issues = repo.get_issues(state = "all")
    
    iter_issues = iter(issues)
    while True:
        try:
            issue = next(iter_issues)
            extracted_data['issues'][issue.id] = {
                'title': issue.title,
                'description': issue.body,
                'labels': [],
                'comments': []
            }
            try:
                iter_labels = iter(issue.labels)
                label = next(iter_labels)
                extracted_data['issues'][issue.id]['labels'].append(label.name)
            except StopIteration:
                continue
            try:
                iter_comments = iter(issue.get_comments())
                comment = next(iter_comments)
                extracted_data['issues'][issue.id]['comments'].append({
                    comment.id: comment.body
                })
            except StopIteration:
                continue
        except gh.RateLimitExceededException:
            logging.warning(g.get_rate_limit())
            sleep_time = g.get_rate_limit().core.reset - dt.datetime.utcnow() + dt.timedelta(0, 20)
            time.sleep()
            continue
        except StopIteration:
            return extracted_data

In [9]:
extracted_data = get_repo_issues("https://github.com/MrpYA45/github-text-mining-tfg")

### Crear una función que permita guardar los datos extraídos
A partir del diccionario que contiene los datos extraídos estos se almacenan en un fichero dentro de la carpeta `/data`

In [10]:
def save_issues(extracted_data: dict) -> None:
    file_name = urlparse(extracted_data['url']).path[1:].replace("/", "_") + "_extracted.json"
    file_path = "data/" + file_name
    with open(file_path, 'w+',  encoding='utf8') as file:
        json.dump(extracted_data, file, indent=4, ensure_ascii=False)

In [11]:
save_issues(extracted_data)

#### Leemos los datos extraídos

In [12]:
with open("data/MrpYA45_github-text-mining-tfg_extracted.json", encoding="utf8") as file:
    data = json.load(file)

pprint(data)

{'issues': {'831250862': {'comments': [{'799201497': 'Recuerda que la guía de '
                                                     'referencia es '
                                                     'https://docs.github.com/en/rest/reference/issues\r\n'
                                                     '@Kencho ¿puedes dejar a '
                                                     'Pablo el código de '
                                                     'extracción de issues y '
                                                     'almacenamiento en BD?'}],
                          'description': 'El objetivo de esta incidencia es '
                                         'investigar como descargar el '
                                         'contenido de las issues de un '
                                         'repositorio mediante la API de '
                                         'GitHub utilizando Python.',
                          'labels': ['investigating'],
   

### Pruebas a mayor escala

In [13]:
import time

Repositiorio de Jizt

In [14]:
start_time = time.time()
extracted_data = get_repo_issues("https://github.com/dmlls/jizt-tfg")
save_issues(extracted_data)
end_time = time.time()
print("Execution time: ", end_time - start_time)

Execution time:  26.405773639678955


Repositorio de PyGithub

In [15]:
start_time = time.time()
extracted_data = get_repo_issues("https://github.com/PyGithub/PyGithub")
save_issues(extracted_data)
end_time = time.time()
print("Execution time: ", end_time - start_time)

Execution time:  215.86081957817078
